In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import branca
#import branca.colormap as cm
import os

import matplotlib as mpl
from shapely.geometry import Point
import geopandas
from geopandas import GeoDataFrame

#import plotly_express as px
import matplotlib.image as mpimg
import matplotlib.colors as mcolors

import random
import seaborn as sns

import math

from matplotlib.patches import Rectangle

from shapely.geometry import Polygon

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

def NGTempCostCurve(Temp,NG_Cost = 3.0, AHF_Coeffs = [0,-0.00038,0.90556]):
    HHV = 54 # MJ/kg
    Density = 0.68 # kg/m3
    cfTom3 = 35.31 # Unit conversion
    AHF = AHF_Coeffs[0]*(int(Temp)^2) + AHF_Coeffs[1]*int(Temp) + AHF_Coeffs[2] # avaialble Heat fraction - Deep Patel Equation
    
    HHV = HHV*Density*(1/cfTom3)*(1/1000000)*(1000) # returns  TJ/thousand cf 
    Cost = NG_Cost*(1/HHV)*(1/AHF)*(1/277.778) # returns the Cost in $/MWh
    return Cost
    

In [8]:

Gens = pd.read_excel('../../SMR_inputs.xlsx',sheet_name='FOAK_act')
Gens = Gens[['Type', 'Power in MWt', 
       'Thermal Efficiency', 'Thermal transfer efficiency',
       'Outlet Temp (C)', 'CAPEX $/MWe', 'FOPEX $/MWe-y',
        'VOM in $/MWh-e', 'Life (y)']]
Gens['Steam temp (C)'] = Gens['Outlet Temp (C)']*Gens['Thermal transfer efficiency']


In [9]:
NF = pd.read_csv('NREL_base_facilities_2.csv', encoding = "ISO-8859-1")


This section will filter down the overall set for the sepcific fuels which you want to use. 
Could make this more robust by:
- Filter out the facilities which have a certain mix (i.e. mostly served by the fuels you are studying rather than just the fuels being used in general
- This could be done by finding the proportion of Total to Natural gas (thoguh would need to be at the facility scale using a groupby function 

In [10]:
NFNG = NF.loc[NF.FUEL_TYPE.isin(['Natural Gas (Weighted U.S. Average)'])]
NFNG.columns

Index(['Unnamed: 0', 'CITY', 'COUNTY', 'COUNTY_FIPS', 'Coal', 'Diesel',
       'END_USE', 'FACILITY_ID', 'FINAL_NAICS_CODE', 'FUEL_TYPE',
       'FUEL_TYPE_BLEND', 'FUEL_TYPE_OTHER', 'LPG_NGL', 'MECS_NAICS',
       'Natural_gas', 'Other', 'Process_byp', 'Pulp_Paper', 'REPORTING_YEAR',
       'Residual_fuel_oil', 'STATE', 'Temp_degC', 'Total', 'UNIT_NAME',
       'UNIT_TYPE', 'for_EU_sum', 'Temp_Band', 'Biogenic', 'MMTCO2E'],
      dtype='object')

Select data for year 2015 only

In [11]:
NFNG2015 = NFNG[NFNG.REPORTING_YEAR ==2015]
NFNG2015.columns

Index(['Unnamed: 0', 'CITY', 'COUNTY', 'COUNTY_FIPS', 'Coal', 'Diesel',
       'END_USE', 'FACILITY_ID', 'FINAL_NAICS_CODE', 'FUEL_TYPE',
       'FUEL_TYPE_BLEND', 'FUEL_TYPE_OTHER', 'LPG_NGL', 'MECS_NAICS',
       'Natural_gas', 'Other', 'Process_byp', 'Pulp_Paper', 'REPORTING_YEAR',
       'Residual_fuel_oil', 'STATE', 'Temp_degC', 'Total', 'UNIT_NAME',
       'UNIT_TYPE', 'for_EU_sum', 'Temp_Band', 'Biogenic', 'MMTCO2E'],
      dtype='object')

Drop rows with Total = 0

In [12]:
NFNG2015.drop(NFNG2015.index[(NFNG2015["Total"] ==0)],axis=0,inplace=True)

In this block, the facilities are 'batched' to the certain temperature thresholds. 
- Could add more thresholds or less depending on your preference
- An example is to create an upper limit at the value of each SMR type rather than the three basic bounds

This output: Facs_total, will also include a total facility scale demand which is servable by the H2. If you wanted, you could output that one (with a 'Temp_Req' of 3000) to run in the H2 model

In [20]:
Gens['Steam temp (C)']
steam_temps = list(Gens['Steam temp (C)'])
steam_temps.sort()
steam_temps

[316.0, 479.4, 630.0, 658.0, 762.45]

In [21]:
Facs_total = pd.DataFrame()

for f in NFNG2015.FACILITY_ID.unique():
    subdf = NFNG2015.loc[NFNG.FACILITY_ID == f]
    temp = pd.DataFrame()
    for t in steam_temps:# [302,630, 700, 750, 950,3000]:
        subdf2 = subdf.loc[subdf.Temp_degC<=t]
        subdf3 = subdf.loc[subdf.Temp_degC>t]
        if len(subdf2.index) >0:
            t_b = ((subdf2.Temp_degC*subdf2.Total)/subdf2.Total.sum()).sum()
            subdf2['Batch_Temp_degC'] = t_b
            subdf2['Highest_Temp_served_degC'] = subdf2.Temp_degC.max()
            subdf2['Emissions_mmtco2/y'] = subdf2.MMTCO2E.sum()
            subdf2['Heat_demand_MWh/hr'] = subdf2.Total.sum()*1.1*277.778/8760
            subdf2['Remaining_Heat_MW'] = (subdf3.Total.sum()*1.1*277.778/8760)
            subdf2['Remaining_temp_degC'] = ((subdf3.Temp_degC*subdf3.Total)/subdf3.Total.sum()).sum()
            subdf2['Temp_Req'] = t
            subdf2['NG_HLMP_mod'] = NGTempCostCurve(t_b,NG_Cost = 1.0)
            subdf2['Helper'] = list(range(len(subdf2.index)))
            temp = pd.concat((temp,subdf2))
    temp.drop_duplicates(subset = ['FACILITY_ID','Batch_Temp_degC'], keep = 'first', inplace = True)
    Facs_total = pd.concat((Facs_total,temp))
Facs_total.reset_index(drop = True, inplace = True)
Facs_total.to_excel('./facs_batched.xlsx', index=False)
print(max(Facs_total['Heat_demand_MWh/hr']))
print(min(Facs_total['Heat_demand_MWh/hr']))

2355.7125117814653
0.0006723813281126485


Here, Full_Data is cretaed which is the SMR results which we discussed previously. It outputs the useful data for you to run the hybrid analysis. 

You can use the 'remianing columns' for the H2 study as well (the orange columns we had on the board)

In [25]:
Full_Data = pd.DataFrame()

for i in Facs_total.index:
    FAC = Facs_total.loc[Facs_total.index == i]
    temp = pd.concat((FAC.loc[FAC.index.repeat(5)].reset_index(drop=True),Gens), axis =1)
    Full_Data = pd.concat((Full_Data,temp.loc[temp['Outlet Temp (C)'] >= temp.Temp_Req]))

Full_Data['Modules'] = Full_Data['Heat_demand_MWh/hr']/(Full_Data['Power in MWt']*0.99)
Full_Data['Modules'] = [math.ceil(x) for x in Full_Data['Modules']]
Full_Data['SMR_Capacity'] = Full_Data['Modules']*Full_Data['Power in MWt']
Full_Data['SMR_Capacity_e'] = Full_Data['SMR_Capacity']*Full_Data['Thermal Efficiency']
Full_Data['Surplus_Capacity'] = (Full_Data['SMR_Capacity']*0.99)-Full_Data['Heat_demand_MWh/hr']
Full_Data['Surplus_Capacity_e'] = Full_Data['Surplus_Capacity']*Full_Data['Thermal Efficiency']
IR = 0.077
Full_Data['CRF'] = (IR/(1-((1+IR)**(-1*(Full_Data['Life (y)'])))))


itc_SMR = 0.3

## FOAK
Full_Data['Total_CAPEX'] = Full_Data['SMR_Capacity_e']*Full_Data['CAPEX $/MWe']
Full_Data['Annual_CAPEX'] = Full_Data['Total_CAPEX']*Full_Data['CRF']*(1-itc_SMR)
Full_Data['FOPEX'] = Full_Data['SMR_Capacity_e']*Full_Data['FOPEX $/MWe-y']
Full_Data['VOPEX'] = (Full_Data['Heat_demand_MWh/hr']*Full_Data['Thermal Efficiency'])*8760*Full_Data['VOM in $/MWh-e']
Full_Data['Annual_Cost'] = Full_Data['Annual_CAPEX'] + Full_Data['FOPEX'] + Full_Data['VOPEX']

## NOAK
"""
Full_Data['Total_CAPEX_NOAK'] = Full_Data['SMR_Capacity_e']*Full_Data['CAPEX $/MWe']
Full_Data['Annual_CAPEX_NOAK'] = Full_Data['Total_CAPEX_NOAK']*Full_Data['CRF']
Full_Data['FOPEX_NOAK'] = Full_Data['SMR_Capacity_e']*Full_Data['FOPEX $/MWe-y']
Full_Data['VOPEX_NOAK'] = (Full_Data['Heat_demand_MWh/hr']*Full_Data['Thermal Efficiency'])*8760*Full_Data['VOM in $/MWh-e']
Full_Data['Annual_Cost_NOAK'] = Full_Data['Annual_CAPEX_NOAK'] + Full_Data['FOPEX_NOAK'] + Full_Data['VOPEX_NOAK']
"""


Full_Data[['FACILITY_ID', 'STATE', 'Heat_demand_MWh/hr','Emissions_mmtco2/y','Temp_Req', 'NG_HLMP_mod',
       'Type', 'Power in MWt', 'SMR_Capacity_e', 'Outlet Temp (C)','Modules','SMR_Capacity','Surplus_Capacity','Annual_Cost']]
    ##'Annual_Cost_NOAK']]

,FACILITY_ID,STATE,Heat_demand_MWh/hr,Emissions_mmtco2/y,Temp_Req,NG_HLMP_mod,Type,Power in MWt,SMR_Capacity_e,Outlet Temp (C),Modules,SMR_Capacity,Surplus_Capacity,Annual_Cost
0,1000588,IN,36.981306,0.053375,316.0,4.135126,PWR,250,77.0,316,1,250,210.518694,8.270059e+05
1,1000588,IN,36.981306,0.053375,316.0,4.135126,HTR,500,265.0,850,1,500,458.018694,3.488229e+06
2,1000588,IN,36.981306,0.053375,316.0,4.135126,MSR,440,195.0,700,1,440,398.618694,1.390842e+06
3,1000588,IN,36.981306,0.053375,316.0,4.135126,MR,12,14.0,700,4,48,10.538694,2.813067e+06
4,1000588,IN,36.981306,0.053375,316.0,4.135126,SFR,286,100.0,510,1,286,246.158694,8.969090e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1011694,IA,13.983712,0.020183,316.0,4.237059,PWR,250,77.0,316,1,250,233.516288,3.802506e+05
1,1011694,IA,13.983712,0.020183,316.0,4.237059,HTR,500,265.0,850,1,500,481.016288,1.427506e+06
2,1011694,IA,13.983712,0.020183,316.0,4.237059,MSR,440,195.0,700,1,440,421.616288,5.962236e+05
3,1011694,IA,13.983712,0.020183,316.0,4.237059,MR,12,7.0,700,2,24,9.776288,1.065683e+06


In [27]:
# NG price
# Merge state-level prices
ng_prices = pd.read_excel('../eia_aeo_industrial_sector_ng_prices.xlsx', sheet_name='state_prices')
# 2024 prices
ng_prices = ng_prices[ng_prices.year == 2024]
ng_prices.rename(columns={'price 2020USD/MMBtu':'NG price ($/MMBtu)'}, inplace=True)
Full_Data= Full_Data.merge(ng_prices, left_on='STATE', right_on='state')


In [28]:
Full_Data['Revenues'] = Full_Data.NG_HLMP_mod*Full_Data['Heat_demand_MWh/hr']*Full_Data['NG price ($/MMBtu)']*8760
Full_Data['SMR Net Ann. Rev. ($/year)'] = Full_Data['Revenues']-Full_Data['Annual_Cost']
Full_Data.to_csv('Full_Spread_SMRs.csv')
Full_Data

,Unnamed: 0,CITY,COUNTY,COUNTY_FIPS,Coal,Diesel,END_USE,FACILITY_ID,FINAL_NAICS_CODE,FUEL_TYPE,...,Total_CAPEX,Annual_CAPEX,FOPEX,VOPEX,Annual_Cost,state,year,NG price ($/MMBtu),Revenues,SMR Net Ann. Rev. ($/year)
0,25113,East Chicago,LAKE,18089,0.0,0.0,CHP and/or Cogeneration Process,1000588,331111,Natural Gas (Weighted U.S. Average),...,1737197.0,94740.506782,13860.0,7.184054e+05,8.270059e+05,IN,2024,5.119415,6.857967e+06,6.030961e+06
1,25113,East Chicago,LAKE,18089,0.0,0.0,CHP and/or Cogeneration Process,1000588,331111,Natural Gas (Weighted U.S. Average),...,2693990.0,146920.572546,27560.0,3.313748e+06,3.488229e+06,IN,2024,5.119415,6.857967e+06,3.369738e+06
2,25113,East Chicago,LAKE,18089,0.0,0.0,CHP and/or Cogeneration Process,1000588,331111,Natural Gas (Weighted U.S. Average),...,1425840.0,77760.210379,35295.0,1.277786e+06,1.390842e+06,IN,2024,5.119415,6.857967e+06,5.467125e+06
3,25113,East Chicago,LAKE,18089,0.0,0.0,CHP and/or Cogeneration Process,1000588,331111,Natural Gas (Weighted U.S. Average),...,253610.0,14411.001986,1834.0,2.796822e+06,2.813067e+06,IN,2024,5.119415,6.857967e+06,4.044900e+06
4,25113,East Chicago,LAKE,18089,0.0,0.0,CHP and/or Cogeneration Process,1000588,331111,Natural Gas (Weighted U.S. Average),...,2215400.0,120819.986867,13100.0,8.835170e+06,8.969090e+06,IN,2024,5.119415,6.857967e+06,-2.111123e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6064,88886,Emmetsburg,PALO ALTO,19147,0.0,0.0,Conventional Boiler Use,1011694,325193,Natural Gas (Weighted U.S. Average),...,1737197.0,94740.506782,13860.0,2.716501e+05,3.802506e+05,IA,2024,4.584279,2.379371e+06,1.999120e+06
6065,88886,Emmetsburg,PALO ALTO,19147,0.0,0.0,Conventional Boiler Use,1011694,325193,Natural Gas (Weighted U.S. Average),...,2693990.0,146920.572546,27560.0,1.253025e+06,1.427506e+06,IA,2024,4.584279,2.379371e+06,9.518652e+05
6066,88886,Emmetsburg,PALO ALTO,19147,0.0,0.0,Conventional Boiler Use,1011694,325193,Natural Gas (Weighted U.S. Average),...,1425840.0,77760.210379,35295.0,4.831684e+05,5.962236e+05,IA,2024,4.584279,2.379371e+06,1.783147e+06
6067,88886,Emmetsburg,PALO ALTO,19147,0.0,0.0,Conventional Boiler Use,1011694,325193,Natural Gas (Weighted U.S. Average),...,126805.0,7205.500993,917.0,1.057560e+06,1.065683e+06,IA,2024,4.584279,2.379371e+06,1.313688e+06


In [29]:
print(len(Full_Data.FACILITY_ID.unique()))

905
